In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
means = [[2, 2], [8, 3], [3, 6]]
cov = [[1, 0], [0, 1]]
N = 500
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X2 = np.random.multivariate_normal(means[2], cov, N)

X = np.concatenate((X0, X1, X2), axis=0)
K = 3

original_label = np.asarray([0]*N + [1]*N + [2]*N).T


Tuần này mình sẽ dùng numpy để xây dựng thuật toán k-means (không dùng thư viện ML như sklearn) 
https://www.youtube.com/watch?v=4b5d3muPQmA.


Ý tưởng là phân dữ liệu thành k cụm. Thuật toán:
Chọn k centroids bất kì
Tính khoảng cách của mỗi điểm tới k điểm centroids, gán điểm đó vào lớp có centroids gần nhất.
Update centroids là trung bình các điểm dữ liệu trong class đấy
Quay lại bước 2 cho tới khi các điểm centroids ở lần lặp tới không đổi.

Viết thuật toán bằng numpy, và vẽ đường phân chia giữa các cluster. Sau đó chạy thuật toán với k=2, 3, 4 với dữ liệu này 
https://colab.research.google.com/drive/1LYM4RPGxygdmpJIdjadIu88qq4ppYtoR?usp=sharing và cho nhận xét.

In [ ]:
def kmeans_display(X, label):
    K = np.amax(label) + 1
    X0 = X[label == 0, :]
    X1 = X[label == 1, :]
    X2 = X[label == 2, :]

    plt.plot(X0[:, 0], X0[:, 1], 'b^', markersize=4, alpha=.8)
    plt.plot(X1[:, 0], X1[:, 1], 'go', markersize=4, alpha=.8)
    plt.plot(X2[:, 0], X2[:, 1], 'rs', markersize=4, alpha=.8)

    plt.axis('equal')
    plt.plot()
    plt.show()


kmeans_display(X, original_label)


In [ ]:
from matplotlib.pyplot import figure
plt.style.use('bmh')
figure(figsize=(18, 16), dpi=80)
kmeans_display(X, original_label)


In [ ]:
def k_cen(X, cluster_nums):
    return X[np.random.choice(X.shape[0], cluster_nums, replace=False)]


k_cen1 = k_cen(X, 3)
k_cen1


In [ ]:
# Tính toán và xác định phần tử thuộc cụm
def cen_cal(X, centers):
    distance11 = cdist(X, centers)
    return np.argmin(distance11, axis=1)


cen_cal(X, k_cen1)


In [ ]:
# Cập nhật lại label
def update_kmeans(X, labels, cluster_nums):
    centers = np.zeros((cluster_nums, X.shape[1]))
    for k in range(cluster_nums):
        new_k = X[labels == k, :]
        centers[k, :] = np.mean(new_k, axis=0)
    return centers


k_cen2 = update_kmeans(X, original_label, 3)
k_cen2


In [ ]:
# Kiểm tra điều kiện dừng
def kmeans_check(cen_1, cen_2):
    return np.array_equal(cen_1, cen_2)


kmeans_check(k_cen1, k_cen2)


In [ ]:
# Visualization
def kmeans_vis(X, centers, labels, cluster_nums):
    fig = plt.gcf()
    fig.set_size_inches(12.5, 8.5, forward=True)

    plt.xlabel('x')
    plt.ylabel('y')
    plt_colors = ['#ff3838', '#ffac26', '#ccff26', '#26ff4a',
                  '#26ffb7', '#26acff', '#0400ff', '#ff00dd', '#ff003c', '#59594c']

    for i in range(cluster_nums):
        data = X[labels == i]
        plt.scatter(data[:, 0], data[:, 1], color=plt_colors[i])
    plt.show()


kmeans_vis(X, k_cen1, original_label, 3)


In [ ]:
# K-means algo
def kmeans(init_centes, X, cluster_nums):
    centers = init_centes
    init_labels = np.zeros(X.shape[0])
    kmeans_vis(X, centers, init_labels, cluster_nums)
    while True:
        labels = cen_cal(X, centers)
        #kmeans_vis(X, centers, labels, cluster_nums)
        new_centers = update_kmeans(X, labels, cluster_nums)
        if kmeans_check(centers, new_centers):
            break
        centers = new_centers
    kmeans_vis(X, centers, labels, cluster_nums)
    return (centers, labels)


kmeans(k_cen1, X, 3)
k_cen2 = k_cen(X, 10)



In [ ]:
kmeans(k_cen2, X, 10)
